In [1]:
!pip install ipywidgets --quiet

In [2]:
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
from matplotlib import cm
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, FloatSlider, fixed

In [3]:
def gradient_descent(x, V, dV, num_dolphins, param, 
                     delta=1e-20, epsilon=1e-20, verbose=False):
    x_old=np.zeros(len(x))
    dV_old=np.zeros(len(x))
    dV_current = dV(x, num_dolphins, param)
    ddV = dV_current - dV_old
    dx = x - x_old
    F_norm = np.inner(ddV,ddV)
    x_norm = la.norm(dx)
    
    step_count=0
    loss_val = [[step_count],[V(x, num_dolphins, param)]]
    
    while x_norm > delta and F_norm > epsilon*epsilon:
        gamma = np.abs(np.inner(dx,ddV))/F_norm
        x_old = np.array(x)
        dV_old = np.array(dV_current)
        x = x - gamma*dV_current
        
        dV_current=dV(x, num_dolphins, param)
        
        ddV = dV_current - dV_old
        dx = x - x_old
        F_norm = np.inner(ddV,ddV)
        x_norm = la.norm(dx)
        step_count += 1
        loss_val[0].append(step_count)
        loss_val[1].append(V(x, num_dolphins, param))
        
        if verbose:
            print(f"Epoch: {step_count} - Cost: {V(x, num_dolphins, param)}...")
            print("---------------------------------------")
    
    print("\n\n")
    if verbose:
        print(f"Descent converges at {x.reshape((num_dolphins, 3))[0, :]}")
        for i in range(1, num_dolphins):
            print(f"                      {x.reshape((num_dolphins, 3))[i, :]}")
        
    return x, loss_val

In [4]:

def cost_function(r, num_dolphins, param):
    
    if not isinstance(r, np.ndarray):
        r = np.array(r)
        
    r = r.reshape((num_dolphins, 3))    
    
    cost = 0
    for i in range(0, num_dolphins):
        for j in range(i, num_dolphins):
            if i==j:
                continue
            else:
                r_ij = ((r[j, 0]-r[i, 0])**2) + ((r[j, 1]-r[i, 1])**2) + ((r[j, 2]-r[i, 2])**2)
                cost += r_ij + 1/r_ij  
            
    return cost


def cost_der(r, num_dolphins, param):  # cost derivative
    
    r = r.reshape((num_dolphins, 3))
    
    derr_array = np.zeros((num_dolphins, 3))
    
    dvdx, dvdy, dvdz = 0, 0, 0
    
    for i in range(0, num_dolphins):
        for j in range (i, num_dolphins):
            if i==j:
                continue
            else:
                r_ij = ((r[j, 0]-r[i, 0])**2) + ((r[j, 1]-r[i, 1])**2) + ((r[j, 2]-r[i, 2])**2)
                dvdx += 2*(r[j, 0] - r[i, 0])*(1-r_ij**2)/(r_ij**2)
                dvdy += 2*(r[j, 1] - r[i, 1])*(1-r_ij**2)/(r_ij**2)
                dvdz += 2*(r[j, 2] - r[i, 2])*(1-r_ij**2)/(r_ij**2)
        
        derr_array[i, 0] = dvdx
        derr_array[i, 1] = dvdy
        derr_array[i, 2] = dvdz
        
        dvdx, dvdy, dvdz = 0, 0, 0
          
    
    derr_array = derr_array.flatten()
    
    return derr_array



In [5]:
# get_pos_n_cost means get position and cost
def get_pos_n_cost(num_dolphins:int=3, verbose=False):

    param=[4.0,8.0,3.0,1.0]

    x = np.random.randint(-10, 10, (num_dolphins, 3)).flatten()
    x_init = x.reshape((num_dolphins, 3))
    
    x_final, loss_val = gradient_descent(x, cost_function, cost_der, num_dolphins, param, verbose=verbose)
    x_final = x_final.reshape((num_dolphins, 3))

    fig, axs = plt.subplots(figsize=(14,8))
    font = {'family': 'serif', 'color': 'black', 'weight': 'bold', 'size': 16}

    axs.plot(loss_val[0], loss_val[1], 'r-', label='Cost')
    axs.set_xlabel('$Epoch$', fontdict=font, labelpad=10)
    axs.set_ylabel('$Cost$ $(r)$', fontdict=font, labelpad=10)
    axs.set_title("Cost as a function of Epochs", fontdict=font)
    axs.legend()
    
    plt.show()
    
    
    fig, axs = plt.subplots(figsize=(14,8))
    font = {'family': 'serif', 'color': 'black', 'weight': 'bold', 'size': 16}
    
    axs = plt.axes(projection='3d')
    
    axs.set_xlim([min(x_final[:, 0].min(), x_init[:, 0].min())  - 0.25, 
                 max(x_final[:, 0].max(), x_init[:, 0].max())  + 0.25])
    axs.set_ylim([min(x_final[:, 1].min(), x_init[:, 1].min()) - 0.25, 
                  max(x_final[:, 1].max(), x_init[:, 1].max()) + 0.25])
    axs.set_zlim([min(x_final[:, 2].min(), x_init[:, 2].min()) - 0.25, 
                  max(x_final[:, 2].max(), x_init[:, 2].max()) + 0.25])
    
    axs.scatter(*x_final.T, marker='o', color='b', linewidths=4, label='Final Dolphin Positions')
    axs.scatter(*x_init.T, marker='*', color='r', linewidths=4,  label='Initial Dolphin Positions')

    axs.set_xlabel(r'x ($\sigma$)', fontdict=font, labelpad=10)
    axs.set_ylabel(r'y ($\sigma$)', fontdict=font, labelpad=10)
    axs.set_zlabel(r'z ($\sigma$)', fontdict=font, labelpad=10)
    axs.set_title("Inital and Final Dolphin Positions", fontdict=font)
    axs.legend()
    plt.show()




In [6]:
# number of dolphins = 3
interact(get_pos_n_cost, num_dolphins=widgets.IntSlider(min=-3.0, max=12.0, step=1, value=3), verbose=False)

interactive(children=(IntSlider(value=3, description='num_dolphins', max=12, min=-3), Checkbox(value=False, de…

<function __main__.get_pos_n_cost(num_dolphins: int = 3, verbose=False)>

In [7]:
# number of dolphins = 4
interact(get_pos_n_cost, num_dolphins=widgets.IntSlider(min=-3.0, max=12.0, step=1, value=4), verbose=False)

interactive(children=(IntSlider(value=4, description='num_dolphins', max=12, min=-3), Checkbox(value=False, de…

<function __main__.get_pos_n_cost(num_dolphins: int = 3, verbose=False)>

In [8]:
# number of dolphins = 6
interact(get_pos_n_cost, num_dolphins=widgets.IntSlider(min=-3.0, max=12.0, step=1, value=6), verbose=False)

interactive(children=(IntSlider(value=6, description='num_dolphins', max=12, min=-3), Checkbox(value=False, de…

<function __main__.get_pos_n_cost(num_dolphins: int = 3, verbose=False)>

In [9]:
# number of dolphins = 8
interact(get_pos_n_cost, num_dolphins=widgets.IntSlider(min=-3.0, max=12.0, step=1, value=8), verbose=False)

interactive(children=(IntSlider(value=8, description='num_dolphins', max=12, min=-3), Checkbox(value=False, de…

<function __main__.get_pos_n_cost(num_dolphins: int = 3, verbose=False)>

In [10]:
# number of dolphins = 10
interact(get_pos_n_cost, num_dolphins=widgets.IntSlider(min=-3.0, max=12.0, step=1, value=10), verbose=False)

interactive(children=(IntSlider(value=10, description='num_dolphins', max=12, min=-3), Checkbox(value=False, d…

<function __main__.get_pos_n_cost(num_dolphins: int = 3, verbose=False)>

### Comments
2b) Gradient descent optimization an optimization algorithm to find the minimum of a function. We start with a random point on the function and move in the negative direction of the gradient of the function to reach the local/global minima.
Above, I provided an interactive widget so we can see that as we increase the number of dolphins, the minimum value of the cost will continue to increase which implies that the cost depends on the number of dolphins.

The cost function is well defined, derivable, and convex since the second derivative of the cost function is greater than 0, the plot for the cost function versus the number of iterations is very steep. However, I do not think the configuration is the optimum result. It only shows that gamma(learning rate) is too high and the implication of this is that the cost function might be converging too quickly to a suboptimal solution. Gamma determines how large of a step should be taken on each iteration in the direction opposite of the gradient (again because we are trying to minimize the cost function). The method may skip over the actual optimal result if we have a larger step.

The model can yield a more optimal solution if gamma is reduced i.e. a smaller learning rate. However, this will significantly increase time complexity by increasing the number of iterations it would take for the cost function to converge. Therefore, we can slowly change gamma until the cost function versus number of iterations curve is smooth exponential decay.



### DISCLAIMER 
#### I USED THE CODES PROVIDED IN THE GRADIENT DESCENT HANDOUT GIVEN IN CLASS BY DR. SAGAR PANDIT
https://machinelearningmastery.com/tour-of-optimization-algorithms/
https://towardsdatascience.com/implement-gradient-descent-in-python-9b93ed7108d1